| Band   | Usage               | A Center| A Width | B Center| B_width | res|
| :--    | :--                 |--:      | --:     | --:     | --:     | --:|
| Band 1 | Coastal aerosol	   |  442.7	 |  21	   |  442.2  |	 21	   | 60 |
| Band 2 | Blue	               |  492.4	 |  66	   |  492.1  |	 66	   | 10 |
| Band 3 | Green	           |  559.8	 |  36	   |  559.0  |	 36	   | 10 |
| Band 4 | Red	               |  664.6	 |  31	   |  664.9  |	 31	   | 10 |
| Band 5 | Vegetation red edge |  704.1	 |  15	   |  703.8  |	 16	   | 20 |
| Band 6 | Vegetation red edge |  740.5	 |  15	   |  739.1  |	 15	   | 20 |
| Band 7 | Vegetation red edge |  782.8	 |  20	   |  779.7  |	 20	   | 20 |
| Band 8 | NIR	               |  832.8	 | 106     |  832.9  |	106	   | 10 |
| Band 8A| Narrow NIR          |  864.7	 |  21	   |  864.0  |	 22	   | 20 |
| Band 9 | Water vapour	       |  945.1	 |  20	   |  943.2  |	 21	   | 60 |
| Band 10| SWIR – Cirrus	   | 1373.5	 |  31	   | 1376.9  |	 30	   | 60 |
| Band 11| SWIR                | 1613.7	 |  91	   | 1610.4  |	 94	   | 20 |
| Band 12| SWIR                | 2202.4	 | 175     | 2185.7  |	185	   | 20 |

In [ ]:
import xarray
import zarr
import pandas
import matplotlib.pyplot as plt
import geopandas
import numpy

In [ ]:
tiles = ['11SLB', # UCSB
         '10SFH', # LNBL 1
         '10SGH', # LNBL 2         
         '11SQD', # DRI 1
         '11SQC', # DRI 2
         '11TNK', # BSU 1
         '11TNJ', # BSU 2
         '11TPK', # BSU 3
         '11TPJ'  # BSU 5
        ]
tile = tiles[2]
tile

In [ ]:
zarr_store = f'/tablespace/sentinel2/{tile}_sharpend.zarr'
ds = xarray.open_zarr(zarr_store)

# Todo
- [x] pansharpening
- [x] Add band coordinates as strings
- [x] slice to UCSB site
- [x] Calculate NDVI + NSDI
- [x] find the time with the lowest NDSI for each
- [x] Read masks: 
    - cloud
    - snow
    - shadow
- [ ] R0 for summer only

# Subset

In [ ]:
def normalize(data):
    data = data - numpy.nanmin(data)
    data = data / numpy.nanmax(data)     
    return data

def mask_nans(data):
    data = numpy.ma.masked_array(data, numpy.isnan(data))
    data[data.mask] = 0
    return data
    
def plot(da, time=None, mask=True):
    fig, ax = plt.subplots(dpi=300)    
        
    if time is not None:
        da = da.sel(time=time)

    rgb = da.squeeze()
                
    if 'band' in da.coords:
        rgb = rgb.transpose('y', 'x', 'band')
    else:
        rgb = rgb.transpose('y', 'x') 
    
    rgb =rgb.values

    if mask:
        rgb = mask_nans(rgb)
        
    img = ax.imshow(rgb)
    ax.axis('off')  
    ax.set_aspect('equal')
    cbar = fig.colorbar(img, ax=ax, shrink=0.7)    
    #return ax

In [ ]:
# Only summer
ds = ds.sel(time=(ds['time.month'] >= 5) & (ds['time.month'] <= 11))
ds

# Calc NDSI and NDVI
NDSI = (B3-B11)/(B3+B11)

In [ ]:
reflectance = ds['reflectance']
# Make sure we don't have 0 in denominator; can happen due to negative reflectances

b8_b4 = (reflectance.sel(band='B8') + reflectance.sel(band='B4'))
b8_b4 = b8_b4.where(b8_b4!=0)
ndvi = (reflectance.sel(band='B8') - reflectance.sel(band='B4')) / b8_b4

b3_b11 = (reflectance.sel(band='B3') + reflectance.sel(band='B11'))
b3_b11 = b3_b11.where(b3_b11!=0)
ndsi = (reflectance.sel(band='B3') - reflectance.sel(band='B11')) / b3_b11

ndsi = ndsi.where(ndsi<1).where(ndsi>-1)
ndvi = ndvi.where(ndvi<1).where(ndvi>-1)

ds['ndvi'] = ndvi
ds['ndsi'] = ndsi

# Plot a random date

In [ ]:
date = ds.time[30]
plot(ds['ndsi'], time=date)

# R0 
## For locations where there exists an observation with NDSI < 0:

1. From those locations, remove all locations which:
    - Have shadow
    - Have snowmask
2. Then, find the observation with the highest NDVI

## For locations where there does not exist an observation with NDSI < 0:
1. Take observations whose blue reflectances is closest, but not smaller than 0.1

In [ ]:
%%time
# this requires each chunk to be read
min_ndsi = ds['ndsi'].min(dim='time')
min_ndsi = min_ndsi.compute() # We can compute; this is only one timeslice and 1 band
plot(min_ndsi)

## SCL keys
| Value | meaning  |
| --:   | :--      |
| 0     | NO_DATA  |
| 1     | SATURATED_OR_DEFECTIVE|
| 2     | CAST_SHADOWS|
| 3     | CLOUD_SHADOWS|
| 4     | VEGETATION|
| 5     | NOT_VEGETATED|
| 6     | WATER|
| 7     | UNCLASSIFIED|
| 8     | CLOUD_MEDIUM_PROBABILITY|
| 9     | CLOUD_HIGH_PROBABILITY|
| 10    | THIN_CIRRUS|
| 11    | SNOW|

We are probably OK with 
- 4: VEGETATION
- 5: NOT_VEGETATED
- 6: WATER
- 7: UNCLASSIFIED

In [ ]:
# 1. Discard all locations that never have a NDSI below 1
candidates = ds.drop_vars(['sun_zenith_grid', 'sun_azimuth_grid', 'viewing_zenith_grid', 'viewing_azimuth_grid' ]).drop_dims(['x_angles', 'y_angles'])
#candidates = ds

ndsi_mask = min_ndsi < 0
candidates = candidates.where(ndsi_mask) 

# 2. Discard all observations when the cloud or snow probablility wasn't 0
candidates = candidates.where(candidates['CLD']==0)
candidates = candidates.where(candidates['SNW']==0)

# 3. Keep only observations that were veg, nonveg, or unclassified
mask_scl = ((candidates['SCL']==4) | (candidates['SCL']==5)  | (candidates['SCL']==7))
candidates = candidates.where(mask_scl)

# 4. Keep only observations where MODIS band 3 (450 nm, +-blue, Sentinel band 2) is larger than 10%
#candidates = candidates.where(candidates['reflectance'].sel(band='B2') > 0.1)

# 5. Keep only observations with negative NDSI
#candidates = candidates.where(candidates['ndsi']<0)

# 6. Keep only observations with more green than blue
candidates = candidates.where(candidates['reflectance'].sel(band='B2') < candidates['reflectance'].sel(band='B3'))

# 7. For all left observations, find for every location the observation with max NDVI
candidates_max_ndvi = candidates['ndvi'].max(dim=('time'))

## Get R0 - via index

In [ ]:
# Replace all nans in NDVI
#masked_ndvi = candidates['ndvi'].where(numpy.isnan(candidates['ndvi']), -2)
masked_ndvi = candidates['ndvi'].fillna(-1)

In [ ]:
%%time
# Find the time indices whith the lowest NDVI
r0_idx = masked_ndvi.argmax(dim='time', skipna=True).compute()

In [ ]:
%%time
# It seems like this is single-core
r0 = ds[['reflectance']].isel(time=r0_idx, drop=True)

In [ ]:
%%time
r0 = r0.compute()

In [ ]:
%%time
zarr_store = f'/tablespace/sentinel2/{tile}_r0.zarr'

r0.to_zarr(zarr_store, mode='w')

# Some plots

In [ ]:
from matplotlib.colors import ListedColormap

fig, ax = plt.subplots(dpi=300)

categories = ['Veg', 'Nonveg', 'Water', 'unclass']
colors = ['blue', 'green', 'red', 'purple']  # Example colors, you can adjust as needed
cmap = ListedColormap(colors)

img = ax.imshow(r0['SCL'].astype(int), cmap=cmap, vmin=4, vmax=7)
#cbar = fig.colorbar(img, ax=ax, ticks=numpy.arange(len(categories)), shrink=0.7)
cbar = fig.colorbar(img, ax=ax, shrink=0.7)    
cbar.ax.set_yticks([4,5,6,7])
cbar.ax.set_yticklabels(categories)

ax.axis('off')  
ax.set_aspect('equal')

In [ ]:
plot(r0['ndsi'], time=None)

In [ ]:
plot(r0['ndvi'], time=None)

In [ ]:
r0_rgb = r0['reflectance'].sel(band=['B4', 'B4', 'B3'])

plot(r0_rgb)

# To geotiff

In [ ]:
r0.rio.write_crs(32611, inplace=True)
r0.rio.set_spatial_dims('x', 'y', inplace=True)
tiff = r0['reflectance'].to_dataset('band')

tiff[['ndsi', 'ndvi']] = r0[['ndsi', 'ndvi']]
tiff = tiff.fillna(-1)
tiff.rio.to_raster('r0.tiff')